In [1]:
from bs4 import BeautifulSoup
import urllib2
import csv

BASE_URL = "http://androidweekly.net/issues/issue-{}"

In [2]:
def contentForIssue(num):
    url = BASE_URL.format(num)
    print url
    request = urllib2.Request(url)
    request.add_header('Accept-Encoding', 'utf-8')
    response = urllib2.urlopen(url)
    soup = BeautifulSoup(response.read().decode('utf-8'), 'lxml')
    content = soup.find('div', { 'class':'issue' })
    return content

In [16]:
def dataForIssue(content, issue_num):
    sections = content.find_all('td')
    last_section_header = ""
    data = []
    section_count = 0
    for section in sections:
        if section.h2:
            section_count += 1
            last_section_header = section.h2.string
        if section.a and section.a.string:
            item = {}
            item['issue_num'] = issue_num
            item['section'] = last_section_header.encode("utf-8")
            item['title'] = section.a.string.encode("utf-8").replace('\n','')
            item['link'] = section.a.get('href').encode("utf-8")
            item['site'] = section.span.string.replace('(','').replace(')', '').replace('\n','').encode("utf-8")
            item['description'] = section.p.string.replace('\n','').encode("utf-8")
            data.append(item)
    print 'Found %d items in %s sections' % (len(data), section_count)
    return data

In [4]:
def writeData(data, fname):
    with open(fname, 'wb') as f:
        w = csv.DictWriter(f, data[0].keys())
        w.writeheader()
        for datum in data:
            w.writerow(datum)

In [5]:
def getAllData(beginning, end):
    allData = []
    for i in reversed(range(beginning, end)):
        content = contentForIssue(i)
        data = dataForIssue(content, i)
        allData += data
        writeData(allData, 'issue_{}.csv'.format(i))
    writeData(allData, 'all_data.csv')

In [17]:
#content = contentForIssue(193)
data = dataForIssue(content, 193)

<td align="left" style="font-family: Arial, sans-serif; vertical-align: top;font-size: 14px; text-align: left; width:80%;" valign="top" width="80%">
<a class="article-headline" href="http://www.thedroidsonroids.com/blog/android/meaningful-motion-with-shared-element-transition-and-circular-reveal-animation/" style="color:#0099CC; text-decoration: none; font-weight: bold;">
Meaningful Motion with Transitions and Circular Reveal
</a>
<span class="main-url" style="color: #333333; font-size: 10px;">
(www.thedroidsonroids.com)
</span>
<p style="margin:0; line-height: 1.5em; font-size: 12px;">
Mariusz Brona shows you a cool transition by using the combination of circular reveal animation, fade animation and shared element transition.
</p>
</td>
test
<p style="margin:0; line-height: 1.5em; font-size: 12px;">
Mariusz Brona shows you a cool transition by using the combination of circular reveal animation, fade animation and shared element transition.
</p>

Mariusz Brona shows you a cool transiti

AttributeError: 'NoneType' object has no attribute 'replace'

In [146]:
getAllData(1, 194)

http://androidweekly.net/issues/issue-215
Found 28 items in 7 sections
http://androidweekly.net/issues/issue-214
Found 21 items in 8 sections
http://androidweekly.net/issues/issue-213
Found 30 items in 7 sections
http://androidweekly.net/issues/issue-212
Found 23 items in 7 sections
http://androidweekly.net/issues/issue-211
Found 29 items in 9 sections
http://androidweekly.net/issues/issue-210
Found 28 items in 9 sections
http://androidweekly.net/issues/issue-209
Found 27 items in 10 sections
http://androidweekly.net/issues/issue-208
Found 25 items in 7 sections
http://androidweekly.net/issues/issue-207
Found 25 items in 8 sections
http://androidweekly.net/issues/issue-206
Found 25 items in 8 sections
http://androidweekly.net/issues/issue-205
Found 34 items in 8 sections
http://androidweekly.net/issues/issue-204
Found 34 items in 10 sections
http://androidweekly.net/issues/issue-203
Found 29 items in 7 sections
http://androidweekly.net/issues/issue-202
Found 29 items in 7 sections
http

AttributeError: 'NoneType' object has no attribute 'replace'

In [138]:
allData[0]

NameError: name 'allData' is not defined